In [1]:
!pip install tensorflow-ranking==0.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 4.0 MB/s eta 0:00:00


In [2]:
!pip install tensorflow==2.9.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 1.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.7 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 703.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.7 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 71.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 75.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 52.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.5.26
    Uninstalling flatbuffers-23.5.26:
      Successfully uninstalled flat

**Importing  libraries**

In [3]:
import pandas as pd
import os
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow_addons.metrics import F1Score
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import regularizers 
from kerastuner.tuners import RandomSearch
import tensorflow_ranking as tfr

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly

### Custom Hamming Loss Metric

In this code  a custom Hamming Loss metric is defined using TensorFlow and Keras. The Hamming Loss is a metric used to evaluate the accuracy of multi-label classification models. Here's an explanation of the code:

#### Custom Metric Class

- A custom metric class named `HammingLoss` is defined. This class extends the `Metric` class provided by TensorFlow/Keras.

- The `__init__` method initializes the metric. It accepts parameters such as `threshold` and `name`. The threshold defines the threshold value for binary conversion of predictions. The `hamming_loss` and `count` variables are created as TensorFlow variables to keep track of the Hamming loss and the number of samples.

#### `update_state` Method

- The `update_state` method is used to update the state of the metric. It accepts `y_true` (true labels), `y_pred` (predicted probabilities), and `sample_weight` (optional).

- The method first converts the predicted probabilities to binary labels based on the specified threshold.

- It then computes the absolute differences between the true labels and the binary predictions for each sample and class.

- The mean over classes for each sample is calculated, which represents the Hamming loss for that sample.

- The Hamming loss and the sample count are updated accordingly.

#### `result` Method

- The `result` method calculates the final Hamming loss by dividing the accumulated Hamming loss by the sample count.

#### `reset_state` Method

- The `reset_state` method is used to reset the Hamming loss and sample count at the end of each epoch. This is important to ensure that the metric calculations are isolated for each epoch.

This custom Hamming Loss metric can be used during the training of multi-label classification models to monitor and evaluate the model's performance with regard to label prediction accuracy.




In [4]:
import tensorflow as tf
from tensorflow.keras.metrics import Metric

class HammingLoss(Metric):
    def __init__(self, threshold=0.5, name="hamming_loss", **kwargs):
        super(HammingLoss, self).__init__(name=name, **kwargs)
        self.threshold = tf.Variable(threshold, trainable=False, dtype=tf.float32)
        self.hamming_loss = self.add_weight(name="hl", initializer="zeros")
        self.count = self.add_weight(name="count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_binary = tf.cast(y_pred > self.threshold, tf.float32)
        y_true = tf.cast(y_true, tf.float32)
        tmp = tf.math.abs(y_true - y_pred_binary)
        hl = tf.math.reduce_mean(tmp, axis=-1)
        self.hamming_loss.assign_add(tf.math.reduce_sum(hl))
        self.count.assign_add(tf.cast(tf.size(y_true) / tf.shape(y_true)[-1], tf.float32))
    def result(self):
        return self.hamming_loss / self.count
    def reset_state(self):
        self.hamming_loss.assign(0.)
        self.count.assign(0.)

### Data Loading and Filtering

This section covers the process of loading data from CSV files and applying filtering to include only specific rows based on the value of the `Disease_Risk` column.

- The data is loaded from the following CSV files:
  - Training data: `train_file.csv`
  - Validation data: `val_file.csv`
  - Test data: `test_file.csv`

- To ensure that the analysis focuses on samples relevant to disease risk, a filter is applied to select rows where the `Disease_Risk` column is equal to 1. This step helps exclude unrelated data.

- Additionally, the disease labels are extracted into the `labels` variable for further reference.

- Finally, information is provided on the sizes of the resulting datasets, including the number of samples in the training, validation, and test sets.

This data preparation process ensures that subsequent analysis or model training is based on the most relevant samples with disease risk.


In [5]:
train_data = pd.read_csv('/kaggle/input/my-data/train_file.csv')
val_data = pd.read_csv('/kaggle/input/my-data/val_file.csv')
test_data = pd.read_csv('/kaggle/input/my-data/test_file.csv')
train_data = train_data[train_data['Disease_Risk'] == 1]
val_data = val_data[val_data['Disease_Risk'] == 1]
test_data = test_data[test_data['Disease_Risk'] == 1]
labels = train_data.columns[2:-1]
len_train_data = len(train_data)
len_val_data = len(val_data)
len_test_data = len(test_data)
filtered_size = len_train_data + len_val_data + len_test_data

len(train_data), len(val_data), len(test_data),filtered_size

(7502, 934, 953, 9389)

### Custom Image Augmentation Layer

In the following code  a custom image augmentation layer is defined using TensorFlow and Keras. This custom layer is designed to apply various image augmentation operations for data preprocessing and augmentation in deep learning models. Here's an explanation of the code:

#### Custom Image Augmentation Class

- The code defines a class called `CustomImageAugmentation` that inherits from `tf.keras.layers.Layer`. This class serves as the foundation for applying image augmentation techniques to input images.

- The class constructor (`__init__`) is responsible for initializing the image augmentation layer. It accepts a set of parameters that allow customization of which augmentation operations are applied. These operations include horizontal flipping, rotation, brightness adjustment, contrast adjustment, saturation adjustment, hue adjustment, scaling, cropping, grid distortion, compression, Gaussian noise, Gaussian blur, downscaling, gamma correction, and elastic transformation.

- The `call` method within this class is used to apply the specified augmentation operations to input images. It includes a parameter named `apply` that controls whether the augmentations should be applied or not.

- The augmentation operations are diverse and encompass random horizontal flipping, random rotation, random brightness, contrast, saturation, and hue adjustments, scaling, and more. The application of each operation is controlled by the corresponding class attribute, such as `self.flip` and `self.rotate`.

- If the `apply` parameter is set to `True`, the augmentation operations are applied to the input images; otherwise, the original images are returned.

- The `img_aug` variable represents an instance of the `CustomImageAugmentation` class, which can be used for data augmentation within image-based deep learning models.

This custom image augmentation layer offers flexibility in specifying and applying image transformations, enhancing model generalization and performance when working with image datasets.


In [6]:
import tensorflow as tf
class CustomImageAugmentation(tf.keras.layers.Layer):
    def __init__(self, flip=True, rotate=True, brightness=True,
                 contrast=True, saturation=True, hue=True, scale=False,
                 crop=False, grid_distortion=False, compression=False,
                 gaussian_noise=False, gaussian_blur=False,
                 downscaling=False, gamma=False, elastic_transform=False, **kwargs):
        super(CustomImageAugmentation, self).__init__(**kwargs)
        self.flip = flip
        self.rotate = rotate
        self.brightness = brightness
        self.contrast = contrast
        self.saturation = saturation
        self.hue = hue
        self.scale = scale
        self.crop = crop
        self.grid_distortion = grid_distortion
        self.compression = compression
        self.gaussian_noise = gaussian_noise
        self.gaussian_blur = gaussian_blur
        self.downscaling = downscaling
        self.gamma = gamma
        self.elastic_transform = elastic_transform

    def call(self, inputs, apply=True):
        if apply:
            augmented = tf.image.random_flip_left_right(inputs) if self.flip else inputs
            augmented = tf.image.rot90(augmented, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)) if self.rotate else augmented
            augmented = tf.image.random_brightness(augmented, max_delta=0.2) if self.brightness else augmented
            augmented = tf.image.random_contrast(augmented, lower=0.5, upper=1.5) if self.contrast else augmented
            augmented = tf.image.random_saturation(augmented, lower=0.5, upper=1.5) if self.saturation else augmented
            augmented = tf.image.random_hue(augmented, max_delta=0.2) if self.hue else augmented
            return augmented
        else:
            return inputs
img_aug = CustomImageAugmentation()

### Custom Image Data Generator for Augmentation

In this code a custom image data generator is defined using TensorFlow and Keras. This data generator is designed to apply various image augmentation operations to preprocess and augment data in deep learning models. Here's an explanation of the code:

#### Custom Image Data Generator Class

- A custom image data generator class is created using TensorFlow and Keras.

- The data generator is configured with various augmentation operations that can be applied during data preprocessing.

- The `preprocessing_function` parameter is defined as a lambda function that utilizes the `img_aug` function with the `apply=True` flag to apply augmentation operations.

- The defined data generator can be used to preprocess and augment images during the training of deep learning models, enhancing the model's ability to learn from diverse and augmented data.




In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    preprocessing_function=lambda x: img_aug(x, apply=True)
)

### Data Generator for Training

In this code a data generator for training deep learning models is defined using TensorFlow and Keras. The data generator is configured to preprocess and augment the training data. Here's an explanation of the code:

#### Data Generator Configuration

- The data generator is configured with the specified `target_size` of (299, 299), which defines the desired size of input images.

- The `batch_size` is set to 32, determining the number of samples to process in each batch during training.

#### Data Flow Configuration

- The `train_generator` is created using the `datagen.flow_from_dataframe` method.

- It is associated with the training data stored in the `train_data` DataFrame.

- The `x_col` parameter specifies the column name in the DataFrame where image file paths are stored.

- The `y_col` parameter is set to the list of column names representing the labels in the DataFrame. These columns are obtained from `train_data.columns[2:-1].tolist()`.

- The `class_mode` is set to 'raw', indicating that the generator should return raw arrays as the target values.

- Images are processed in batches of size `batch_size`.

- The `target_size` parameter is set to the specified dimensions of (299, 299) for image resizing.

- Data shuffling is enabled with the `shuffle` parameter set to `True`.

This data generator is essential for efficiently feeding training data to deep learning models, enabling data augmentation and resizing to match model input requirements.


In [8]:
target_size=(299, 299)
batch_size=32
train_generator = datagen.flow_from_dataframe(
    dataframe=train_data,
    x_col='IMG_DIR',
    y_col=labels,
    class_mode='raw',
    batch_size=batch_size,
    target_size=target_size,
    shuffle=True
)

Found 7502 validated image filenames.


In [9]:
val_generator = datagen.flow_from_dataframe(
    dataframe=val_data,
    x_col='IMG_DIR',
    y_col=labels,
    class_mode='raw',
    batch_size=batch_size,
    target_size=target_size,
    shuffle=False  
)

Found 934 validated image filenames.


In [10]:
for batch_images, batch_labels in train_generator:
    print("Batch images shape:", batch_images.shape)
    print("Batch labels shape:", batch_labels.shape)
    break

Batch images shape: (32, 299, 299, 3)
Batch labels shape: (32, 28)


### Hyperparameter-Tuned Model Builder

In this code a model builder is defined for hyperparameter tuning of an image classification model. The model architecture is based on the InceptionV3 with custom dense layers and various hyperparameters.

#### Model Architecture

- The code utilizes TensorFlow's Keras API to create a neural network model for image classification.
- The base model used is InceptionV3, pre-trained on ImageNet, with the option to fine-tune its layers.
- Global Average Pooling (GAP) is applied to the output of the base model to reduce the spatial dimensions.
- The architecture of the dense layers, their units, and L2 regularization are hyperparameters tuned during the process.
- The model includes three dense layers, each with customizable units and L2 regularization terms.
- A Dropout layer with a tunable rate is introduced for regularization.
- The output layer consists of multiple neurons, typically equal to the number of classes (specified as 'len(labels)'). A sigmoid activation function is used for multi-label classification.

#### Hyperparameter Tuning

- The `model_builder` function is designed to be used with a hyperparameter tuning framework (e.g., Keras Tuner).
- The function takes hyperparameters as inputs, allowing the architecture and regularization to be customized during the tuning process.
- Various hyperparameters, such as dense units, L2 regularization, and dropout rate, are declared and tunable within the function.

#### Compilation

- The model is compiled with the Adam optimizer and specific learning rate.
- Binary cross-entropy is chosen as the loss function, as it is suitable for binary multi-label classification.
- Multiple metrics are defined, including accuracy, AUC, precision, recall, F1-score, Hamming loss, and Mean Average Precision (mAP), suitable for multi-label classification problems.

This code provides a flexible and tunable model builder for multi-label image classification tasks, allowing for the optimization of various architectural and regularization hyperparameters.


In [11]:
def model_builder(hp):
    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=img_shape)
        base_model.trainable = True
        inputs = Input(shape=img_shape)
        x = base_model(inputs, training=False)
        x = tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)
        x = GlobalAveragePooling2D()(x)


        hp_units_1 = hp.Int('dense_units_1', min_value=512, max_value=2048, step=256, default=1024)
        hp_units_2 = hp.Int('dense_units_2', min_value=256, max_value=1024, step=128, default=512)
        hp_units_3 = hp.Int('dense_units_3', min_value=128, max_value=512, step=64, default=256)

        hp_l2_1 = hp.Float('l2_1', min_value=1e-6, max_value=1e-2, sampling='log', default=1e-4)
        hp_l2_2 = hp.Float('l2_2', min_value=1e-6, max_value=1e-2, sampling='log', default=1e-4)
        hp_l2_3 = hp.Float('l2_3', min_value=1e-6, max_value=1e-2, sampling='log', default=1e-4)

        x = Dense(
            hp_units_1,
            activation='relu',
            kernel_regularizer=regularizers.l2(hp_l2_1)  
        )(x)


        x = Dense(
            hp_units_2,
            activation='relu',
            kernel_regularizer=regularizers.l2(hp_l2_2) 
        )(x)


        x = Dense(
            hp_units_3,
            activation='relu',
            kernel_regularizer=regularizers.l2(hp_l2_3) 
        )(x)

        x = Dropout(hp.Float('dropout_rate', min_value=0.3, max_value=0.7, step=0.1, default=0.5))(x)


        predictions = Dense(len(labels), activation='sigmoid')(x)

        model = keras.Model(inputs=inputs, outputs=predictions)
        model.compile(optimizer=Adam(learning_rate=0.00001),
                  loss="binary_crossentropy",
                  metrics=['accuracy',tf.keras.metrics.AUC(name="auc",  multi_label=True,num_labels=len(labels)),
                             tf.keras.metrics.AUC(name="auc_roc", curve="ROC", multi_label=True,num_labels=len(labels)),
                            tf.keras.metrics.AUC(name="auc_pr", curve="PR", multi_label=True,num_labels=len(labels)),
                            tf.keras.metrics.Precision(name="precision"),
                            tf.keras.metrics.Recall(name="recall"),
                            F1Score(num_classes=len(labels),average='weighted',threshold=0.5),
                           HammingLoss(),
                            tfr.keras.metrics.MeanAveragePrecisionMetric(name="map")])
        return model

In [12]:
img_shape=(299,299,3)

### Hyperparameter Tuning Configuration

In this code the configuration for hyperparameter tuning of a neural network model is defined using the Keras Tuner library.

#### Hyperparameter Tuner

- The code imports the Keras Tuner library as `kt` for consistency.
- A hyperparameter tuner is created using the `RandomSearch` method, which performs a random search over a defined hyperparameter space.
- The tuner is set to search for hyperparameters that optimize the "val_f1_score," with the goal of maximizing this metric.

#### Objective and Trials

- The objective of the tuning is set to maximize the "val_f1_score," indicating that the F1 score on the validation dataset is the primary metric of interest.
- The `max_trials` parameter is set to 5, determining the number of trials to run during hyperparameter optimization. Adjust this value based on the computational resources available.

#### Tuning Directory

- The directory for storing tuning-related information is specified as 'hyperparameter_tuning'.
- The project name is designated as 'custom_image_augmentation' for easy reference.

This code sets up the configuration for hyperparameter tuning, enabling the search for optimal model hyperparameters while focusing on maximizing the F1 score on the validation dataset.


In [13]:
import keras_tuner as kt  
tuner = RandomSearch(
    model_builder,
    objective=kt.Objective("val_f1_score", direction="max"), 
    max_trials=5,  
    directory='hyperparameter_tuning',
    project_name='custom_image_augmentation'
)

87910968/87910968 [==============================] - 0s 0us/step


In [14]:
tuner

### Learning Rate Adjustment Callback

In this code  a custom callback class named `LRA` is defined for learning rate adjustment during the training of a deep learning model. The primary purpose of this callback is to dynamically adjust the learning rate based on various conditions, optimizing the training process. Here's an explanation of the code:

#### Callback Class

- The `LRA` callback class is designed to adjust the learning rate during training. It accepts multiple parameters such as initial learning rate, patience, stop patience, threshold, factor, dwell, batches, initial epoch, and total epochs.

#### Learning Rate Adjustment

- The callback monitors training and validation metrics, such as accuracy, F1 score, validation loss, and more. Depending on specified criteria, the learning rate is adjusted accordingly.

- The callback tracks factors like the improvement of validation accuracy, validation F1 score, and the reduction of validation loss.

- If specified thresholds are not met or improvement is lacking, the callback reduces the learning rate and continues training. The process can be customized with dwell functionality to revert to better points in the optimization space.

- If adjustments continue without substantial improvement within the defined stop patience, the training process is halted.

The `LRA` callback offers dynamic control over the learning rate, ensuring that the deep learning model converges effectively and efficiently during training. It serves as a valuable tool for fine-tuning the model's performance.




In [15]:

class LRA(keras.callbacks.Callback):
    def __init__(self,initial_lr, patience,stop_patience, threshold, factor, dwell, batches, initial_epoch,epochs):
        super(LRA, self).__init__()
        self.patience=patience 
        self.stop_patience=stop_patience 
        self.threshold=threshold 
        self.factor=factor 
        self.dwell=dwell
        self.batches=batches 
        self.initial_epoch=initial_epoch
        self.epochs=epochs
        self.count=0 
        self.stop_count=0   
        self.highest_f1_score=0
        self.best_epoch=1         
        self.initial_lr=initial_lr         
        self.highest_tracc=0.0 
        self.lowest_vloss=np.inf 
    def on_epoch_end(self, epoch, logs=None):  
        lr=float(tf.keras.backend.get_value(self.model.optimizer.lr)) 
        current_lr=lr
        v_loss=logs.get('val_loss')  
        acc=logs.get('accuracy')  
        loss=logs.get('loss')
        auc=logs.get('auc')
        v_auc=logs.get('val_auc')
        Map=logs.get('map')
        val_map=logs.get('val_map')
        v_acc = logs.get('val_accuracy')
        val_precision = logs.get('val_precision')
        val_recall = logs.get('val_recall')
        val_f1_score = logs.get('val_f1_score')
        val_hamming_loss = logs.get('val_hamming_loss')
        precision = logs.get('precision')
        recall = logs.get('recall')
        f1_score = logs.get('f1_score')
        hamming_loss = logs.get('hamming_loss')
        auc_roc=logs.get('auc_roc')
        val_auc_roc=logs.get('val_auc_roc')
        auc_pr=logs.get('auc_pr')
        val_auc_pr=logs.get('val_auc_pr')
        if v_acc < self.threshold: 
            monitor='val_accuracy'
            if epoch ==0:
                pimprov=0.0
            else:
                pimprov= (v_acc-self.highest_tracc )*100/self.highest_tracc
            if v_acc>self.highest_tracc: 
                self.highest_tracc=v_acc 
                self.best_weights=self.model.get_weights() 
                self.count=0 
                self.stop_count=0 
                if v_loss<self.lowest_vloss:
                    self.lowest_vloss=v_loss
                if val_f1_score>self.highest_f1_score :
                    self.highest_f1_score = val_f1_score
                color= (0,255,0)
                self.best_epoch=epoch + 1              
            else: 
                if self.count>=self.patience -1: 
                    color=(245, 170, 66)
                    lr= lr* self.factor 
                    tf.keras.backend.set_value(self.model.optimizer.lr, lr) 
                    self.count=0 
                    self.stop_count=self.stop_count + 1 
                    self.count=0 # reset counter
                    if self.dwell:
                        self.model.set_weights(self.best_weights)                     
                    else:
                        if v_loss<self.lowest_vloss:
                            self.lowest_vloss=v_loss    
                        if val_f1_score>self.highest_f1_score :
                            self.highest_f1_score = val_f1_score
                else:
                    self.count=self.count +1   
        elif val_f1_score < 0.8:
            monitor='val_f1_score'
            if epoch ==0:
                pimprov=0.0
            else:
                pimprov= (val_f1_score-self.highest_f1_score )*100/self.highest_f1_score
            if val_f1_score>self.highest_f1_score: 
                self.highest_f1_score=val_f1_score 
                self.best_weights=self.model.get_weights() 
                self.count=0 
                self.stop_count=0 
                if v_loss<self.lowest_vloss:
                    self.lowest_vloss=v_loss
                if v_acc>self.highest_tracc:
                    self.highest_tracc= v_acc
                color= (0,255,0)
                self.best_epoch=epoch + 1          
            else: 
                if self.count>=self.patience -1: 
                    color=(245, 170, 66)
                    lr= lr* self.factor 
                    tf.keras.backend.set_value(self.model.optimizer.lr, lr) 
                    self.count=0 
                    self.stop_count=self.stop_count + 1 
                    self.count=0 
                    if self.dwell:
                        self.model.set_weights(self.best_weights)                        
                    else:
                        if v_loss<self.lowest_vloss:
                            self.lowest_vloss=v_loss  
                        if v_acc>self.highest_tracc:
                            self.highest_tracc= v_acc
                else:
                    self.count=self.count +1           
        else: 
            monitor='val_loss'
            if epoch ==0:
                pimprov=0.0
            else:
                pimprov= (self.lowest_vloss- v_loss )*100/self.lowest_vloss
            if v_loss< self.lowest_vloss: 
                self.lowest_vloss=v_loss          
                self.best_weights=self.model.get_weights() 
                self.count=0 
                self.stop_count=0  
                color=(0,255,0)                
                self.best_epoch=epoch + 1 
            else: 
                if self.count>=self.patience-1: 
                    color=(245, 170, 66)
                    lr=lr * self.factor                   
                    self.stop_count=self.stop_count + 1 
                    self.count=0 
                    tf.keras.backend.set_value(self.model.optimizer.lr, lr) 
                    if self.dwell:
                        self.model.set_weights(self.best_weights) 
                else: 
                    self.count =self.count +1                    
                if v_acc>self.highest_tracc:
                    self.highest_tracc= v_acc
                if val_f1_score>self.highest_f1_score :
                    self.highest_f1_score = val_f1_score
        
        if self.stop_count> self.stop_patience - 1: 
            msg=f' training has been halted at epoch {epoch + 1} after {self.stop_patience} adjustments of learning rate with no improvement'
            print_in_color(msg, (0,255,255), (55,65,80))
            self.model.stop_training = True 

In [16]:
train_steps=int(np.ceil(len(train_generator.labels)/batch_size))
train_steps

235

In [17]:
working_dir = '/kaggle/working/'

### Training Configuration Parameters

This section outlines key parameters used to configure the training process of a deep learning model:

#### `epochs`

- `epochs`: The total number of training epochs. In this example, it is set to 20. Each epoch represents one complete iteration through the entire training dataset.

#### `patience`

- `patience`: This parameter determines the number of epochs to wait before considering adjustments to the learning rate if the monitored value (e.g., accuracy) does not improve. Here, it is set to 8.

#### `stop_patience`

- `stop_patience`: Specifies the number of epochs to wait before halting the training if the monitored value does not exhibit improvement. A value of 2 means that if the model's performance does not improve over the course of 2 consecutive epochs, the training process will be stopped.

#### `threshold`

- `threshold`: The threshold is a critical value that impacts which aspect of the model's performance is monitored. If the training accuracy falls below this threshold (0.65 in this example), the callback focuses on monitoring the training accuracy. If the training accuracy is equal to or exceeds this threshold, the callback switches to monitoring the validation loss.

#### `factor`

- `factor`: Represents the rate at which the learning rate is reduced when an adjustment is triggered. A factor of 0.1 signifies that the learning rate will be reduced to one-tenth of its previous value.

#### `dwell`

- `dwell`: When set to `True`, this parameter enables an experimental feature. If the monitored metric does not improve in the current epoch, the model's weights will be reset to those of the previous epoch, aiming to explore better training outcomes.

#### `ask_epoch`

- `ask_epoch`: Specifies the number of epochs to run before prompting the user or training process to inquire about halting training. It is set to 100 in this example.

#### `batches`

- `batches`: Defines the number of training batches to process per epoch. The specific value may depend on your dataset and hardware setup.

#### `csv_path`

- `csv_path`: Represents the path where the training-related CSV file will be saved. It's defined by joining the `working_dir` and 'my_csv'.

These training configuration parameters are essential for controlling the training dynamics, ensuring efficient learning, and monitoring the model's performance.


In [18]:
epochs =20
patience= 8 
stop_patience =2 
threshold=.65 
factor=0.1 
dwell=True 
ask_epoch=100 
batches=train_steps
csv_path=os.path.join(working_dir,'my_csv')


### Custom Colorful Text Printer

In this code a custom text printer function called `print_in_color` is defined. This function enables the printing of text messages in a user-specified foreground color on a custom background color. Here's an explanation of the code:

#### Custom Text Printer Function

- The `print_in_color` function is designed to display text in a customized color scheme.

- It accepts three parameters:

  - `txt_msg`: The text message to be printed.
  - `fore_tupple`: A tuple representing the RGB values of the foreground color.
  - `back_tupple`: A tuple representing the RGB values of the background color.

- Within the function, an ANSI escape code is constructed to specify the desired text and background colors using the RGB values provided in the tuples.

- The `print` function is used to display the `txt_msg` with the specified colors.

- After printing, a reset code (`'\33[0m'`) is included to return the print color to the default black.

This `print_in_color` function serves as a helpful tool for enhancing the visual appeal of text in code output or command line interfaces by incorporating custom colors.



In [19]:
import time
import datetime
from datetime import datetime
def print_in_color(txt_msg,fore_tupple,back_tupple,):

    rf,gf,bf=fore_tupple
    rb,gb,bb=back_tupple
    msg='{0}' + txt_msg
    mat='\33[38;2;' + str(rf) +';' + str(gf) + ';' + str(bf) + ';48;2;' + str(rb) + ';' +str(gb) + ';' + str(bb) +'m' 
    print(msg .format(mat), flush=True)
    print('\33[0m', flush=True) 
    return

### Custom Callback for Clearing Training Output

In this code  a custom callback class named `ClearTrainingOutput` is defined. This callback is intended to clear the training output from the console window once the training is complete. Here's an explanation of the code:

#### Custom Callback Class

- The `ClearTrainingOutput` class extends the functionality of the `tf.keras.callbacks.Callback` class to define custom behavior when the training ends.

- Within the `on_train_end` method, the code checks the operating system (OS) using `os.name`. If the OS is Windows (indicated by `'nt'`), the `cls` command is executed to clear the console. For non-Windows OS (e.g., Unix-like systems), the `clear` command is used for the same purpose.

- This clearing action ensures that the console window is free from the training output, providing a cleaner and more organized environment.

After defining this custom callback, it is used when conducting a hyperparameter search using the Keras Tuner.



In [20]:

import os
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        os.system('cls' if os.name == 'nt' else 'clear')
tuner.search(
    train_generator,
    validation_data=val_generator,
    callbacks=[LRA(initial_lr=0.00001,patience=patience,stop_patience=stop_patience, threshold=threshold,
                   factor=factor,dwell=dwell, batches=batches,initial_epoch=0,epochs=epochs), ClearTrainingOutput()], epochs=epochs,
    validation_steps=None, shuffle=False, initial_epoch=0
)




Trial 5 Complete [01h 01m 28s]
val_f1_score: 0.15849845111370087

Best val_f1_score So Far: 0.2259935587644577
Total elapsed time: 05h 06m 53s


### Display Best Hyperparameters

In this code the best hyperparameters obtained from the hyperparameter tuning process are retrieved and displayed. Here's an explanation of the code:

#### Retrieving Best Hyperparameters

- The `tuner.get_best_hyperparameters(num_trials=1)` method is used to obtain the best hyperparameters. With `num_trials=1`, we retrieve the top-performing hyperparameters.

#### Displaying Best Hyperparameters

- The code prints the best hyperparameters, including values for `Dense Units 1`, `Dense Units 2`, `Dense Units 3`, `L2 Regularization 1`, `L2 Regularization 2`, `L2 Regularization 3`, and `Dropout Rate`.

- These hyperparameters are crucial for configuring the model architecture and training settings. Displaying them allows us to understand the optimal settings found during hyperparameter tuning.

By showcasing the best hyperparameters, they can be used to build and train the final deep learning model with the most effective configuration.


In [21]:

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best Hyperparameters:")
print(f"Dense Units 1: {best_hps.get('dense_units_1')}")
print(f"Dense Units 2: {best_hps.get('dense_units_2')}")
print(f"Dense Units 3: {best_hps.get('dense_units_3')}")
print(f"L2 Regularization 1: {best_hps.get('l2_1')}")
print(f"L2 Regularization 2: {best_hps.get('l2_2')}")
print(f"L2 Regularization 3: {best_hps.get('l2_3')}")
print(f"Dropout Rate: {best_hps.get('dropout_rate')}")

Best Hyperparameters:
Dense Units 1: 512
Dense Units 2: 896
Dense Units 3: 384
L2 Regularization 1: 2.5803151973980912e-05
L2 Regularization 2: 4.289969209505223e-06
L2 Regularization 3: 0.0017392802275116102
Dropout Rate: 0.3
